<a href="https://colab.research.google.com/github/monjjjjj/DecisionTreeClassifier/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix


In [ ]:
data = pd.read_csv("car.csv", names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'target'])
#We can check the first five samples of the data:
data.head(5)

,buying,maint,doors,persons,lug_boot,safety,target
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [ ]:
X = data.drop(columns=['target'])
X = np.array(X.values)
y = np.array(data['target'].values)
label = np.unique(y)
label1 = set(y)
print(label)
print(label1)
for i in range(len(y)):
    for j in range(len(label)):
         if y[i] == label[j]:
            y[i] = j
            break
y = y.astype('int')
print(X[0])
print(X[:,1])

['acc' 'good' 'unacc' 'vgood']
{'good', 'acc', 'unacc', 'vgood'}
['vhigh' 'vhigh' '2' '2' 'small' 'low']
['vhigh' 'vhigh' 'vhigh' ... 'low' 'low' 'low']


In [ ]:
print(len(X[:,1]))
print(len(X[1,:]))

1728
6


In [ ]:
for i in range(len(X[1, :])):
  for 


In [ ]:
import math
probs = []
total = len(y)
for n in range(len(label)):
  probs = np.append(probs, np.sum(y == n))
probs = probs / total

for prob in probs:
    entropy = np.sum(-prob*math.log(prob, 2))
print(entropy)

In [ ]:
data.columns

In [ ]:
data.isnull().sum()

In [ ]:
data["unacc"].value_counts()

In [ ]:
import sklearn.metrics
from sklearn import datasets
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
import numpy as np
import pandas as pd

DEFAULT = None, None


class Node:
    "Decision tree node"

    def __init__(self, entropy, num_samples, num_samples_per_class, predicted_class, num_errors, alpha=float("inf")):
        self.entropy = entropy  # the entropy of current node
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class  # the majority class of the split group
        self.feature_index = 0  # the feature index we used to split the node
        self.threshold = 0  # for binary split
        self.left = None  # left child node
        self.right = None  # right child node
        self.num_errors = num_errors  # error after cut
        self.alpha = alpha  # each node alpha


class DecisionTreeClassifier:
    def __init__(self, max_depth=4):
        self.max_depth = max_depth

    def _entropy(self, sample_y, n_classes):
        # TODO: calculate the entropy of sample_y and return it
        # sample_y represent the label of node
        # entropy = -sum(pi *log2(pi))
        entropy = 0
        probs = []
        total = len(sample_y)

        for n in range(n_classes):
            if len(sample_y[sample_y == n]) != 0 and len(sample_y) != 0:
                probs = np.append(probs, np.sum(sample_y == n))
            else:
                entropy = 0

        probs = probs / total

        for prob in probs:
            entropy = np.sum(-prob * math.log(prob, 2))

        return entropy

    def _feature_split(self, X, y, n_classes):
        # Returns:
        #  best_idx: Index of the feature for best split, or None if no split is found.
        #  best_thr: Threshold to use for the split, or None if no split is found.

        m = y.size

        if m <= 1:
            return DEFAULT

        # Entropy of current node.
        best_criterion = self._entropy(y, n_classes)

        best_idx, best_thr = DEFAULT
        # TODO: find the best split, loop through all the features, and consider all the
        # midpoints between adjacent training samples as possible thresholds.
        # Compute the Entropy impurity of the split generated by that particular feature/threshold
        # pair, and return the pair with smallest impurity.
        best = 0
        for i in range(len(X[0])):
            data = X[:, i]
            thr_list = []

            for j in range(len(np.unique(data)) - 1):
                thr_list.append((sorted(np.unique(data))[j] + sorted(np.unique(data))[j + 1]) / 2)

            for thr in thr_list:
                rate = len(y[data <= thr]) / len(y)
                info = rate * self._entropy(y[data <= thr], n_classes) + (1 - rate) * self._entropy(y[data > thr],
                                                                                                    n_classes)
                if best < best_criterion - info:
                    best = best_criterion - info
                    best_idx = i
                    best_thr = thr

        return best_idx, best_thr

    def _build_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        correct_label_num = num_samples_per_class[predicted_class]
        num_errors = y.size - correct_label_num
        node = Node(
            entropy=self._entropy(y, self.n_classes_),
            num_samples=y.size,
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
            num_errors=num_errors
        )

        if depth < self.max_depth:
            idx, thr = self._feature_split(X, y, self.n_classes_)
            node.feature_index = idx
            node.threshold = thr
            if idx is not None:
                # TODO: Split the tree recursively according index and threshold until maximum depth is reached.
                node.left = self._build_tree(X[X[:, idx] <= thr], y[X[:, idx] <= thr], depth + 1)
                node.right = self._build_tree(X[X[:, idx] > thr], y[X[:, idx] > thr], depth + 1)
                pass
        return node

    def fit(self, X, Y):
        # TODO
        # Fits to the given training data
        self.n_classes_ = len(np.unique(Y))
        self.tree_ = self._build_tree(X, Y)
        pass

    def predict(self, X):
        pred = []
        # TODO: predict the label of data
        for i in X[:]:
            node = self.tree_
            while node.left is not None and node.right is not None:
                if i[node.feature_index] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            pred.append(node.predicted_class)
        return pred

    def _find_leaves(self, root):
        # TODO
        ## find each node child leaves number
        node_list = [root]
        leaf = 0

        while len(node_list) != 0:
            node = node_list[-1]
            node_list.pop()
            if node.left == None and node.right == None:
                leaf = leaf + 1
            else:
                node_list.append(node.right)
                node_list.append(node.left)
        return leaf

    def _error_before_cut(self, root):
        # TODO
        ## return error before post-pruning
        node_list = [root]
        error = 0

        while len(node_list) != 0:
            node = node_list[-1]
            node_list.pop()
            if node.left is None and node.right is None:
                error = error + node.num_errors
            else:
                node_list.append(node.right)
                node_list.append(node.left)
        return error

    def _compute_alpha(self, root):
        # TODO
        ## Compute each node alpha
        # alpha = (error after cut - error before cut) / (leaves been cut - 1)
        node_list = [root]
        while len(node_list) != 0:
            node = node_list[-1]
            node_list.pop()
            leaf = self._find_leaves(node)
            if leaf > 1:
                node.alpha = (node.num_errors - self._error_before_cut(node)) / (leaf - 1)

            if node.left is not None and node.right is not None:
                node_list.append(node.right)
                node_list.append(node.left)
        pass

    def _find_min_alpha(self, root):
        MinAlpha = float("inf")
        # TODO
        ## Search the Decision tree which have minimum alpha's node
        node_list = [root]

        while len(node_list) != 0:
            node = node_list[-1]
            node_list.pop()

            if self._find_leaves(node) > 1:
                if MinAlpha > node.alpha:
                    MinAlpha = node.alpha

            if node.left is not None and node.right is not None:
                node_list.append(node.right)
                node_list.append(node.left)
        return MinAlpha

    def _prune(self):
        # TODO
        # prune the decision tree with minimum alpha node
        node_list = [self.tree_]
        self._compute_alpha(self.tree_)
        cut_node = self._find_min_alpha(self.tree_)

        while len(node_list) != 0:
            node = node_list[-1]
            node_list.pop()

            if self._find_leaves(node) > 1:
                if cut_node == node.alpha:
                    node.left = None
                    node.right = None
                    return
            if node.left is not None and node.right is not None:
                node_list.append(node.right)
                node_list.append(node.left)
        pass


def load_train_test_data(test_ratio=.3, random_state=1):
    df = pd.read_csv('./car.data', names=['buying', 'maint',
                                          'doors', 'persons', 'lug_boot', 'safety', 'target'])
    X = df.drop(columns=['target'])
    X = np.array(X.values)
    y = np.array(df['target'].values)
    label = np.unique(y)

    # label encoding
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i] == label[j]:
                y[i] = j
                break
    y = y.astype('int')

    # data encoding
    for i in range(X.shape[1]):
        data = np.unique(X[:, i])
        for j in range(X.shape[0]):
            for n in range(len(data)):
                if X[j, i] == data[n]:
                    X[j, i] = n
                    break
    X = X.astype('int')

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_ratio, random_state=random_state, stratify=y)
    return X_train, X_test, y_train, y_test


def accuracy_report(X_train_scale, y_train, X_test_scale, y_test, max_depth=7):
    tree = DecisionTreeClassifier(max_depth=max_depth)
    tree.fit(X_train_scale, y_train)
    pred = tree.predict(X_train_scale)

    print(" tree train accuracy: %f"
          % (sklearn.metrics.accuracy_score(y_train, pred)))
    pred = tree.predict(X_test_scale)
    print(" tree test accuracy: %f"
          % (sklearn.metrics.accuracy_score(y_test, pred)))

    for i in range(10):
        print("=============Cut=============")
        tree._prune()
        pred = tree.predict(X_train_scale)
        print(" tree train accuracy: %f"
              % (sklearn.metrics.accuracy_score(y_train, pred)))
        pred = tree.predict(X_test_scale)
        print(" tree test accuracy: %f"
              % (sklearn.metrics.accuracy_score(y_test, pred)))


def main():
    X_train, X_test, y_train, y_test = load_train_test_data(test_ratio=.3, random_state=1)
    accuracy_report(X_train, y_train, X_test, y_test, max_depth=8)


if __name__ == "__main__":
    main()


 tree train accuracy: 0.718776
 tree test accuracy: 0.655106
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.655106
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.655106
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.655106
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.655106
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.653179
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.653179
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.653179
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.653179
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.655106
=============Cut=============
 tree train accuracy: 0.718776
 tree test accuracy: 0.658960
